#Variational Quantum Eigensolver(VQE)、QAOAセミナー 2-6（組合せ最適化問題編4）

2-5-5 Quantum Alternating Operator Ansatz
上記はなかなか答えが出ませんでした。ここでは、正答率を大きく上げる方法を確認します。ハミルトニアンの制約条件を定式化から外すできます。前述の交通最適化問題などは３つのルートから１つのルートを選ぶという制約式が正答率を下げる原因でしたが、今回はその制約式を減らすテクニックを確認します。

XX+YYゲートを導入することで、01と10を入れ替える操作をできます。時間発展を見ても、入れ替え操作が見れます。最初から00と11をださず、01と10のみで計算をします。

<img src="https://qiita-user-contents.imgix.net/https%3A%2F%2Fqiita-image-store.s3.ap-northeast-1.amazonaws.com%2F0%2F88256%2F0b4a9d82-8ebf-8191-23d7-dbb76396e9d1.png?ixlib=rb-1.2.2&auto=format&gif-q=60&q=75&w=1400&fit=max&s=616a52dfc97ba36e01aba646f9affdac">

引用：https://qiita.com/gyu-don/items/c51a9e3d5d16a6d5baf6

これらの操作を導入して簡単な問題を解いてみます。




##2-5-6 Quantum Alternating Operator Ansatzの例題
今回は初期状態に|10>と|01>のもつれ状態を採用し、途中のmixerにXYミキサーというXX+YYの時間発展を導入したものを利用してみます。

In [1]:
!pip install blueqat

     |████████████████████████████████| 51kB 1.7MB/s 


In [1]:
import numpy as np
from blueqat import Circuit
from blueqat.pauli import X, Y, Z, I
from blueqat.pauli import qubo_bit as q
from blueqat.vqe import AnsatzBase, Vqe

def an(index):
    return 0.5 * X[index] + 0.5j * Y[index]

def cr(index):
    return 0.5 * X[index] - 0.5j * Y[index]

op = (cr(1) * an(0) + cr(0) * an(1)).to_expr().simplify()

class QaoaQaoaAnsatz(AnsatzBase):
    def __init__(self, hamiltonian, step=1, init_circuit=None):
        super().__init__(hamiltonian, step * 2)
        self.hamiltonian = hamiltonian.to_expr().simplify()
        if not self.check_hamiltonian():
            raise ValueError("Hamiltonian terms are not commutable")

        self.step = step
        self.n_qubits = self.hamiltonian.max_n() + 1
        if init_circuit:
            self.init_circuit = init_circuit
            if init_circuit.n_qubits > self.n_qubits:
                self.n_qubits = init_circuit.n_qubits
        else:
            self.init_circuit = Circuit(self.n_qubits).h[:]
        self.init_circuit.make_cache()
        self.time_evolutions = [term.get_time_evolution() for term in self.hamiltonian]
        self.time_evolutions2 = [term.get_time_evolution() for term in op]
    def check_hamiltonian(self):
        """Check hamiltonian is commutable. This condition is required for QaoaAnsatz"""
        return self.hamiltonian.is_all_terms_commutable()

    def get_circuit(self, params):
        c = self.init_circuit.copy()
        betas = params[:self.step]
        gammas = params[self.step:]
        for beta, gamma in zip(betas, gammas):
            beta *= np.pi
            gamma *= 2 * np.pi
            for evo in self.time_evolutions:
                evo(c, gamma)
            for evo2 in self.time_evolutions2:
                evo2(c, beta)
        return c

h = -3*q(0)-3*q(1)-2*q(0)*q(1)
h = h.to_expr().simplify()
runner = Vqe(QaoaQaoaAnsatz(h,4,Circuit().h[0].cx[0,1].x[0]))
result = runner.run()

# get probability
print(result.most_common(12))

print('Result by QAOA')
print(runner.ansatz.get_energy_sparse(result.circuit))

# Hamiltonian to matrix
mat = h.to_matrix()

# Calculate by numpy
print('Result by numpy')
print(np.linalg.eigh(mat)[0][0])

(((0, 1), 0.4999999999999996), ((1, 0), 0.49999999999999944), ((1, 1), 1.7333369499485123e-31), ((0, 0), 5.623715437610729e-32))
Result by QAOA
-2.9999999999999973
Result by numpy
-8.0


通常、最小基底状態は-8を選びますが、上記は|01>と|10>のもつれ状態を利用しているため、制約条件をハード制約としてもち、|00>と|11>をとることができないため、QAOAで得られる最小値の期待値は-3となっています。

このように、通常ソフト制約として出てくるはずのハード制約をかすことで、正答率を大幅に上げることができます。回路自体は長くなっています。

<img width="1281" alt="スクリーンショット 2020-02-25 0.39.32.png" src="https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/218694/85efc267-c34d-dbc8-1bb0-25bc22cfb1c0.png">

このように、通常ソフト制約として出てくるはずのハード制約をかすことで、正答率を大幅に上げることができます。回路自体は長くなっています。

<img width="1281" alt="スクリーンショット 2020-02-25 0.39.32.png" src="https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/218694/85efc267-c34d-dbc8-1bb0-25bc22cfb1c0.png">

##2-5-7 量子もつれを使った制約条件項なしの交通最適化計算
ケーススタディとして交通最適を取り上げます。
量子アニーリングでは探索はほぼランダムで行われます。初期状態は|+>からスタートし、途中過程の探索は$\sigma_x$の横磁場を使います。

しかしここでは、ルート選択は１つだけを選ぶわけですから、$q_0$か$q_1$の片方は1で、もう片方は0となるのが条件としてついてきます。また、$q_2$と$q_3$も同様です。

ここで、量子回路でまず量子もつれを作ります。量子もつれは多くのパターンから限られた組み合わせだけが出るように調整できるのでした。簡単な例として、

In [2]:
from blueqat import Circuit
Circuit().h[0].cx[0,1].m[:].run(shots=100)

Counter({'11': 53, '00': 47})

こうすると、本来４種類出るところから00と11のみ答えが出ます。


In [3]:
from blueqat import Circuit
Circuit().h[0].cx[0,1].x[0].m[:].run(shots=100)

Counter({'01': 59, '10': 41})

ちょっと形を変えることで、01と10のもつれに変更できます。

これを使います。つまり、自動車１と自動車２にそれぞれもつれを活用して、01と10に制限をかけるように初期状態を設定します。

今回４量子ビットの回路では、

In [4]:
from blueqat import Circuit
Circuit().h[0].cx[0,1].x[0].h[2].cx[2,3].x[2].m[:].run(shots=100)

Counter({'0110': 33, '1001': 24, '0101': 19, '1010': 24})

###入れ替え作業
$\sigma_x$の横磁場での探索の代わりに明示的に(XX+YY)/2というゲートを利用することで、01と10を入れ替えることができます。

これにより、初期状態で準備した01と10をひたすら入れ替えながら、量子断熱計算で、ハミルトニアンの最小値を求めるQAOAと組み合わせることができます。

###具体的コード
具体的な実行コードはこちらです。

In [5]:
import numpy as np
from blueqat import Circuit
from blueqat.pauli import X, Y, Z, I
from blueqat.pauli import qubo_bit as q
from blueqat.vqe import AnsatzBase, Vqe

def an(index):
    return 0.5 * X[index] + 0.5j * Y[index]

def cr(index):
    return 0.5 * X[index] - 0.5j * Y[index]

op1 = (cr(1) * an(0) + cr(0) * an(1)).to_expr().simplify()
op2 = (cr(3) * an(2) + cr(2) * an(3)).to_expr().simplify()

class QaoaQaoaAnsatz(AnsatzBase):
    def __init__(self, hamiltonian, step=1, init_circuit=None):
        super().__init__(hamiltonian, step * 2)
        self.hamiltonian = hamiltonian.to_expr().simplify()
        if not self.check_hamiltonian():
            raise ValueError("Hamiltonian terms are not commutable")

        self.step = step
        self.n_qubits = self.hamiltonian.max_n() + 1
        if init_circuit:
            self.init_circuit = init_circuit
            if init_circuit.n_qubits > self.n_qubits:
                self.n_qubits = init_circuit.n_qubits
        else:
            self.init_circuit = Circuit(self.n_qubits).h[:]
        self.init_circuit.make_cache()
        self.time_evolutions = [term.get_time_evolution() for term in self.hamiltonian]
        self.time_evolutions1 = [term.get_time_evolution() for term in op1]
        self.time_evolutions2 = [term.get_time_evolution() for term in op2]
    def check_hamiltonian(self):
        """Check hamiltonian is commutable. This condition is required for QaoaAnsatz"""
        return self.hamiltonian.is_all_terms_commutable()

    def get_circuit(self, params):
        c = self.init_circuit.copy()
        betas = params[:self.step]
        gammas = params[self.step:]
        for beta, gamma in zip(betas, gammas):
            beta *= np.pi
            gamma *= 2 * np.pi
            for evo in self.time_evolutions:
                evo(c, gamma)
            for evo1 in self.time_evolutions1:
                evo1(c, beta)
            for evo2 in self.time_evolutions2:
                evo2(c, beta)
        return c

h = 4*q(0)+4*q(1)+4*q(2)+4*q(3)+4*q(0)*q(1)+4*q(0)*q(2)+8*q(1)*q(2)+2*q(1)*q(3)+2*q(2)*q(3)
h = h.to_expr().simplify()
runner = Vqe(QaoaQaoaAnsatz(h,4,Circuit().h[0].cx[0,1].x[0].h[2].cx[2,3].x[2]))
result = runner.run()

# get probability
print(result.most_common(12))

print('Result by QAOA')
print(runner.ansatz.get_energy_sparse(result.circuit))

(((1, 0, 0, 1), 0.8112133792184058), ((0, 1, 0, 1), 0.09121447195335018), ((1, 0, 1, 0), 0.06340958241714689), ((0, 1, 1, 0), 0.03416256641109033), ((0, 0, 0, 1), 1.409502470243749e-31), ((1, 0, 0, 0), 6.548161810916602e-32), ((1, 1, 1, 0), 5.105576867286511e-32), ((1, 1, 0, 1), 4.7792585722809673e-32), ((0, 1, 0, 0), 3.8518598887744717e-32), ((1, 0, 1, 1), 9.485204976107137e-33), ((0, 0, 1, 0), 8.288258806869007e-33), ((0, 1, 1, 1), 1.3601880232234853e-33))
Result by QAOA
8.709367804863955


とても高い確率で制約条件を満たしながら混雑の最も少ないルートが選択されています。
